1. Import Libraries

In [ ]:
import requests
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error
from datetime import datetime, timedelta
import pytz

In [ ]:
API_KEY = '21276c37dacd7a3bce7bb37ba07855dd'
BASE_URL = 'https://api.openweathermap.org/data/2.5/' #base url for making API requests

2. Fetch Current weather data

In [ ]:
def get_current_weather(city):
  url = f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric"
  response = requests.get(url)
  data = response.json()
  return {
      "latitude": data["coord"]["lat"],
      "longitude": data["coord"]["lon"],
      "city": data["name"],
      "current_temp": round(data["main"]["temp"]), #celcius
      "feels_like": round(data["main"]["feels_like"]),#celcius
      "temp_min": round(data["main"]["temp_min"]),#celcius
      "temp_max": round(data["main"]["temp_max"]),#celcius
      "humidity": round(data["main"]["humidity"]),#percentage
      "wind_speed": data["wind"]["speed"],#m/s
      "description": data["weather"][0]["description"],
      "country": data["sys"]["country"],

  }

In [ ]:
get_current_weather("Nashik")

{'latitude': 19.9833,
 'longitude': 73.8,
 'city': 'Nashik',
 'current_temp': 36,
 'feels_like': 34,
 'temp_min': 36,
 'temp_max': 36,
 'humidity': 13,
 'wind_speed': 1.32,
 'description': 'clear sky',
 'country': 'IN'}

3. Read Historical Data

In [ ]:
def read_historical_data(filename):
  df = pd.read_csv(filename)
  df = df.dropna()
  df = df.drop_duplicates()
  return df

In [ ]:
read_historical_data("weather.csv")

,MinTemp,MaxTemp,WindGustDir,WindGustSpeed,Humidity,Pressure,Temp,RainTomorrow
0,8.0,24.3,NW,30.0,29,1015.0,23.6,Yes
1,14.0,26.9,ENE,39.0,36,1008.4,25.7,Yes
2,13.7,23.4,NW,85.0,69,1007.2,20.2,Yes
3,13.3,15.5,NW,54.0,56,1007.0,14.1,Yes
4,7.6,16.1,SSE,50.0,49,1018.5,15.4,No
...,...,...,...,...,...,...,...,...
361,9.0,30.7,NNW,76.0,15,1010.8,30.0,No
362,7.1,28.4,N,48.0,22,1016.9,28.2,No
363,12.5,19.9,ESE,43.0,47,1022.8,18.3,No
364,12.5,26.9,NW,46.0,39,1016.2,25.9,No


4. Prepare data for training

In [ ]:
def prepare_data(data):
  le = LabelEncoder()
  data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
  data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'])

  #define feature and target variables
  X = data[['MinTemp', 'MaxTemp', 'WindGustSpeed', 'Humidity', 'Temp']]
  y = data['RainTomorrow']

  return X, y, le

5. Train Rain Prediction Model

In [ ]:
def train_rain_model(X, y):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  model = RandomForestClassifier(n_estimators=100, random_state=42)
  model.fit(X_train, y_train)

  y_pred = model.predict(X_test)
  accuracy = accuracy_score(y_test, y_pred)
  MSE = mean_squared_error(y_test, y_pred)
  print(f"Accuracy: {accuracy}")
  print(f"Mean Squared Error: {MSE}")

  return model

6. Prepare Regression Data(temperature, humidity, etc.)

In [ ]:
def prepare_regression_data(data, feature):
  X, y = [], [] #initialize list for feature and target values

  for i in range(len(data) - 1):
    X.append(data[feature].iloc[i])
    y.append(data[feature].iloc[i + 1])

  X = np.array(X).reshape(-1, 1)
  y = np.array(y)

  return X, y


7. Train Regression Model

In [ ]:
def train_regression_model(X, y):
  model = RandomForestRegressor(n_estimators=100, random_state=42)
  model.fit(X, y)

  return model

8. Predicting Future Values

In [ ]:
def predict_future(model, current_value):
  predictions = [current_value]

  for i in range(5):
    next_value = model.predict(np.array([[predictions[-1]]]))
    predictions.append(next_value[0])

  return predictions[1:]

9. Weather Analysis Function

In [ ]:
from time import timezone
def weather_view():
  city = input("Enter city name: ")
  current_weather = get_current_weather(city)

  #Loading historical data.
  historical_data = read_historical_data('/content/weather.csv')

  #Prepare and train the rain prediction model.
  X, y, le = prepare_data(historical_data)

  rain_model = train_rain_model(X, y)

  current_data = {

      'MinTemp': current_weather['temp_min'],
      'MaxTemp': current_weather['temp_max'],

      #'Precipitaion': current_weather['precipitation'],
      'WindGustSpeed': current_weather['wind_speed'],
      'Humidity': current_weather['humidity'],
      'Temp': current_weather['current_temp'],
  }

  current_df = pd.DataFrame([current_data])

  #Rain prediction.
  rain_prediction = rain_model.predict(current_df)[0]

  #Preparing regression model for Temperature and Humidity.
  X_temp, y_temp = prepare_regression_data(historical_data, 'Temp')
  X_humidity, y_humidity = prepare_regression_data(historical_data, 'Humidity')

  temp_model = train_regression_model(X_temp, y_temp)
  humidity_model = train_regression_model(X_humidity, y_humidity)

  #Predicting future Temperature and Humidity.
  future_temp = predict_future(temp_model, current_weather['current_temp'])
  future_humidity = predict_future(humidity_model, current_weather['humidity'])

  #Preparing time for future predictions.
  timezone = pytz.timezone('Asia/Kolkata')
  current_time = datetime.now(timezone)
  next_hour = current_time + timedelta(hours=1)
  next_hour = next_hour.replace(minute=0, second=0, microsecond=0)

  future_times = [(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(1, 6)]

  #Display results.
  print(f"City: {city}, {current_weather['city']}")
  print(f"Current Temperature: {current_weather['current_temp']}°C")
  print(f"Feels Like: {current_weather['feels_like']}°C")
  print(f"Min Temperature: {current_weather['temp_min']}°C")
  print(f"Max Temperature: {current_weather['temp_max']}°C")
  print(f"Current Humidity: {current_weather['humidity']}%")
  print(f"Weather Prediction: {current_weather['description']}")
  print(f"Rain Prediction: {'Yes' if rain_prediction else 'No'}")

  print("\nFuture Temperature Predictions:")

  for time, temp in zip(future_times, future_temp):
    print(f"{time}: {round(temp, 1)}°C")

  print("\nFuture Humidity Predictions:")

  for time, humidity in zip(future_times, future_humidity):
    print(f"{time}: {round(humidity, 1)}%")


weather_view()


Enter city name: Jalgaon
Accuracy: 0.863013698630137
Mean Squared Error: 0.136986301369863
City: Jalgaon, Jalgaon
Current Temperature: 39°C
Feels Like: 36°C
Min Temperature: 39°C
Max Temperature: 39°C
Current Humidity: 10%
Weather Prediction: clear sky
Rain Prediction: No

Future Temperature Predictions:
18:00: 33.1°C
19:00: 25.1°C
20:00: 20.1°C
21:00: 16.6°C
22:00: 14.4°C

Future Humidity Predictions:
18:00: 44.8%
19:00: 46.2%
20:00: 48.2%
21:00: 53.7%
22:00: 58.1%
